In [114]:
import tensorflow.keras as keras
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt


keras.utils.set_random_seed(253)

In [115]:
def yajuns_function(t: np.float32):
    return(-1 * np.power(np.e, t)) + np.power(np.e, 2*t) 

t_test = np.arange(0, 5, .01, dtype='float32')
y_test = np.array([yajuns_function(t) for t in t_test], dtype='float32')


In [116]:
# residual is how accurate the output is with respect to target equation
def calc_residual(model: keras.Model, alpha: tf.Variable, beta: tf.Variable, t_vals)-> tf.Tensor:
    
    t_vals_tensor = tf.convert_to_tensor(t_vals, dtype='float32')


    with tf.GradientTape(persistent= True) as gt:

        gt.watch(t_vals_tensor)

        y = model(t_vals_tensor)

        # print(t_vals_tensor)
        
        # predicted values
        y_prime = gt.gradient(y, t_vals_tensor)
        
        y = tf.convert_to_tensor(tf.squeeze(y), dtype='float32')

    y_dprime = gt.gradient(y_prime, t_vals_tensor) # second derivative of predicted values
    del gt

    # print("\n")
    # print("t: "+ str(type(t_vals_tensor)))
    # print("y: " + str(y))
    # print("dt: " + str(tf.reduce_mean(y_prime)))
    # print("d2t: " + str(y_dprime))
    # print("residual: " + str(tf.reduce_mean(tf.add(y_dprime, tf.add(tf.scalar_mul(alpha, y_prime), tf.scalar_mul(beta, y))))))
    # print("\n")
    
    return tf.add(y_dprime, tf.add(tf.scalar_mul(alpha, y_prime), tf.scalar_mul(beta, y)))


In [117]:
# calculate how accurate the model is with respect to the actual answers
def calc_loss(model: keras.Model, alpha: tf.Variable, beta: tf.Variable, t_vals):
    """
    Custom loss function made for neural network. It calculates the residual, and then
    adds it to the difference in the boundary outputs and predicted values.
    Returns the loss.
    
    """

    # need residual for loss, it will tell us how accurate vals are with respect to itself
    residual = calc_residual(model, alpha, beta, t_vals)  
    av_residual = tf.reduce_mean(tf.square(residual))

    t = tf.convert_to_tensor(np.array([0.0], dtype = 'float32'))
    with tf.GradientTape() as gt:
        gt.watch(t)
        y = model(t) # y(0) should equal 0
        
    dy_dt = gt.gradient(y, t) # y'(0) should equal 1    

    
    # print("\n")
    # print("average residual: " + str(av_residual))
    # print("y: " + str(y))
    # print("dy_dt: " + str(dy_dt))
    # print("dy_dt -1: " + str(dy_dt - 1))
    # print("returns: " + str(tf.add(av_residual, tf.add(tf.pow(y, 2), tf.pow(tf.subtract(dy_dt, 1), 2)))))
    # print("\n")
    
    

    return tf.add(av_residual, tf.add(tf.square(y), tf.square(tf.subtract(dy_dt, 1)))) # residual should equal 0, y should equal 0, and dt should equal 1 

In [118]:
# gradient is of the loss function with respect to all the variables in the model
def calc_gradient(model: keras.Model, alpha: tf.Variable, beta: tf.Variable, t_vals):
    """
    Calculates the gradient to apply to the neural network. Returns the
    gradient.
    
    """

    with tf.GradientTape() as gt:
        gt.watch(model.trainable_variables) # weights
        loss = calc_loss(model, alpha, beta, t_vals)

    gradient = gt.gradient(loss, model.trainable_variables)

    # print(str(gradient))

    return loss, gradient

In [119]:
def train(model: keras.Model, alpha: tf.Variable, beta: tf.Variable, t_vals, optimizer: keras.optimizers.Optimizer):
    """
    Executes one training step for the network. Calculates loss and gradient, applies the 
    gradient, then returns the loss.
    
    """
    
    loss, gradient = calc_gradient(model, alpha, beta, t_vals)

    optimizer.apply_gradients(zip(gradient, model.trainable_variables))

    return loss

In [120]:
# make layers of the model
# most accurate 2 period wave is
# 20, 15, 10, 5 with .5 dropout in between

def make_network() -> tuple:
    layers = []         

    layers.append(keras.layers.Dense(16, activation = 'elu', input_shape = (1,)))
    layers.append(keras.layers.Dense(16, activation = 'elu'))
    layers.append(keras.layers.Dense(16, activation = 'elu'))

    output_layer = keras.layers.Dense(1, activation = 'linear', name = "output")
    layers.append(output_layer)

    output_layer.add_weight(shape = (), dtype='float32', trainable=True, name = 'alpha')
    output_layer.add_weight(shape = (), dtype='float32', trainable=True, name = 'beta')
    
    network = keras.Sequential(layers)

    #print(output_layer.trainable_weights)

    return (network, output_layer)


In [121]:
# set up batches
batch_size = 1000000

t_batches = []

i = 0
while i < (len(t_test) / batch_size):
    t_batches.append(np.array(t_test[i * batch_size : (i * batch_size) + batch_size], dtype='float32'))
    i += 1


In [122]:
# train the model
epochs = 1000
i = 0
network, output_layer = make_network()
alpha = output_layer.trainable_weights[0]
beta = output_layer.trainable_weights[1]

while i < epochs:

    loss = 0
    j = 0

    while j < len(t_batches): # train each batch per epoch
        loss += train(network, alpha, beta, t_batches[j], keras.optimizers.Adam())
        j += 1


    print("Epoch: " + str(i))
    print("Loss: " + str(loss / j))
    print("Alpha: " + str(alpha))
    print("Beta: " + str(beta))
    print("\n")
    # save every epoch so i can jump ship at any time and still have a network to show for it
    network.save("alpha_beta_problem.weights.h5")

    i += 1

tf.Tensor(
[0.         0.01       0.02       0.03       0.04       0.05
 0.06       0.07       0.08       0.09       0.09999999 0.11
 0.12       0.13       0.14       0.14999999 0.16       0.17
 0.17999999 0.19       0.19999999 0.21       0.22       0.22999999
 0.24       0.25       0.26       0.26999998 0.28       0.29
 0.29999998 0.31       0.32       0.32999998 0.34       0.35
 0.35999998 0.37       0.38       0.39       0.39999998 0.41
 0.42       0.42999998 0.44       0.45       0.45999998 0.47
 0.48       0.48999998 0.5        0.51       0.52       0.53
 0.53999996 0.55       0.56       0.57       0.58       0.59
 0.59999996 0.61       0.62       0.63       0.64       0.65
 0.65999997 0.66999996 0.68       0.69       0.7        0.71
 0.71999997 0.72999996 0.74       0.75       0.76       0.77
 0.78       0.78999996 0.79999995 0.81       0.82       0.83
 0.84       0.84999996 0.85999995 0.87       0.88       0.89
 0.9        0.90999997 0.91999996 0.93       0.94       0.95
 0.96  

KeyboardInterrupt: 

In [ ]:
network, output_layer = make_network()
network.load_weights("alpha_beta_problem.weights.h5", skip_mismatch=True)
x = np.arange(0, 5, .01, dtype='float32')
t = np.arange(0, 5, .01, dtype='float32')
# y = list(map(lambda a: a*10,y)) # magnification :(


predictions = []

for number in t:
    prediction = network.predict(tf.scalar_mul(number, x), verbose = 0)
    y = np.squeeze(prediction)

    # y = [2000 * number * np.sin(item) for item in x]

    
    predictions.append(y)


plt.ion()

figure = plt.figure()
sub1 = figure.add_subplot(111)

line1, = sub1.plot(t_test, y_test, 'b-')
line2, = sub1.plot(x, predictions[0], "r--")

# for item in predictions:

#     line2.set_ydata(item)

#     figure.canvas.draw()

for item in predictions:
    line2.set_ydata(item)
    display(figure)
    clear_output(wait=True)
    plt.pause(.2)
    

KeyboardInterrupt: 